# TorchSDE + Neuromancer: Basic (Explicit) Integration of Stochastic Differential Equations

This notebook goes over how to utilize torchsde's functionality within Neuromancer framework. This notebook is based off: https://github.com/google-research/torchsde/blob/master/examples/demo.ipynb




### Imports

If necessary, install torchsde library

In [1]:
#!pip install torchsde

In [2]:
import torch
from neuromancer.psl import plot
from neuromancer import psl
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from neuromancer.system import Node, System
from neuromancer.dynamics import integrators, ode
from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.loggers import BasicLogger
from neuromancer.dataset import DictDataset
from neuromancer.constraint import variable
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks

torch.manual_seed(0)


### Dataset: 
Below we generate some data. We assume the process has a state size of 1, batch size of 5 and 100 timesteps. 

***Note: For visualization purposes, we recommend ONLY to use a state size of 1***

In [13]:
batch_size, state_size, t_size = 5, 1, 100
ts = torch.linspace(0, 1, t_size)
y0 = torch.full(size=(batch_size, state_size), fill_value=0.1)
y0.shape
my_data = {'y': y0, 't':ts}

Strongly recommended to refer to TorchSDE documentation. For the basic/explicit SDE case, TorchSDE requires user to define the drift (f) and diffusion (g) functions. We define an example f and g below: 

In [4]:
def f(t, y):
    return torch.sin(t) + 0.1 * y
    
def g(t, y):
    return 0.3 * torch.sigmoid(torch.cos(t) * torch.exp(-y))

### Neuromancer Integration: 
Now define Neuromancer variables and components

In [5]:
t = variable('t')
y = variable('y')
block = blocks.BasicSDE(f, g, t, y) #We use this block for the basic/explicit SDE case where f and g are defined 
integrator = integrators.BasicSDEIntegrator(block) #instantiate integrator for the basic/explicit case 
model = Node(integrator, input_keys=['y','t'], output_keys=['ys']) #define Neuromancer Node to wrap integrator. Output of the 

### Visualization 
Visualize ys which represents ts number of samples from a stochasic process parameterized by f and g for the provided batch_size

In [6]:
# Plotting helper function
def plot(ts, samples, xlabel, ylabel, title=''):
    ts = ts.cpu()
    samples = samples.squeeze().t().cpu()
    plt.figure()
    for i, sample in enumerate(samples):
        plt.plot(ts, sample, marker='x', label=f'sample {i}')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()